In [1]:
#if running locally dont run this cell
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AG5feYaWB5bh1CzI3hHrlBZSaEaRtAfFNKwDaULQkz8-uNVkMVpmlQ
Mounted at /content/drive


In [2]:
loc = '/content/drive/My Drive/data_amazon/subdir/pairs_Category.txt'

In [3]:
import pandas as pd
import numpy as np

In [4]:
pairs_Category = pd.read_csv(loc, sep="-", header=None)
pairs_Category.columns = ["reviewerID", "reviewHash"]
pairs_Category.drop([0],inplace=True)
print('len ',len(pairs_Category))
# pred - category - column 3
pairs_Category.head()

len  14000


,reviewerID,reviewHash
1,U281659737,R934811302
2,U670561057,R657711680
3,U433746872,R750304163
4,U327816997,R865011815
5,U323131234,R222729968


In [5]:
len(pairs_Category),pairs_Category.shape

(14000, (14000, 2))

In [6]:
train = pd.read_csv('/content/drive/My Drive/data_amazon/subdir/train.csv')
print('lenth of train ', len(train))
train.categoryID.unique()

lenth of train  199992


array([0, 4, 3, 1, 2])

In [7]:
train.head()

,reviewTime,reviewText,helpful,reviewerID,reviewHash,categories,unixReviewTime,itemID,rating,summary,categoryID
0,2013-09-26,The model in this picture has them rolled up a...,"{'nHelpful': 0, 'outOf': 0}",U490934656,R798569390,"[['Clothing, Shoes & Jewelry', 'Women'], ['Clo...",1380153600,I402344648,4.0,High Waisted,0
1,2013-02-07,"I love the look of this bra, it is what I want...","{'nHelpful': 0, 'outOf': 0}",U714157797,R436443063,"[['Clothing, Shoes & Jewelry', 'Women', 'Cloth...",1360195200,I697650540,4.0,Beautiful but size runs small,0
2,2014-03-16,I am not comfortable with wearing my wedding b...,"{'nHelpful': 0, 'outOf': 0}",U507366950,R103439446,"[['Clothing, Shoes & Jewelry', 'Wedding Party ...",1394928000,I464613034,5.0,Great Alternative for Nurses,0
3,2014-03-10,Like the look of this top and really looks cut...,"{'nHelpful': 0, 'outOf': 0}",U307862152,R486351639,"[['Clothing, Shoes & Jewelry', 'Women', 'Cloth...",1394409600,I559560885,2.0,One size fits all...Questionable,0
4,2013-07-30,I'm quite small and the XS fits me like a regu...,"{'nHelpful': 1, 'outOf': 1}",U742726598,R508664275,"[['Clothing, Shoes & Jewelry', 'Women', 'Plus-...",1375142400,I476005312,5.0,Great shirt,0


In [8]:
train.categories.unique()

array(["[['Clothing, Shoes & Jewelry', 'Women'], ['Clothing, Shoes & Jewelry', 'Novelty, Costumes & More', 'Novelty', 'Clothing', 'Women', 'Leggings']]",
       "[['Clothing, Shoes & Jewelry', 'Women', 'Clothing', 'Lingerie, Sleep & Lounge', 'Intimates', 'Bras', 'Everyday Bras'], ['Clothing, Shoes & Jewelry', 'Women', 'Petite', 'Intimates', 'Bras', 'Everyday Bras']]",
       "[['Clothing, Shoes & Jewelry', 'Wedding Party Gifts', 'Gifts for the Groomsmen'], ['Clothing, Shoes & Jewelry', 'Women'], ['Clothing, Shoes & Jewelry', 'Novelty, Costumes & More', 'Novelty']]",
       ...,
       "[['Clothing, Shoes & Jewelry', 'Shoes & Accessories: International Shipping Available'], ['Clothing, Shoes & Jewelry', 'M', 'Minnetonka'], ['Clothing, Shoes & Jewelry', 'Women', 'Shoes', 'Flats'], ['Clothing, Shoes & Jewelry', 'Women', 'Shoes', 'Loafers & Slip-Ons']]",
       "[['Clothing, Shoes & Jewelry', 'Women', 'Jewelry', 'Fine', 'Earrings', 'Stud'], ['Clothing, Shoes & Jewelry', 'Women', 'Jewelry',

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199992 entries, 0 to 199991
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   reviewTime      199992 non-null  object 
 1   reviewText      199991 non-null  object 
 2   helpful         199992 non-null  object 
 3   reviewerID      199992 non-null  object 
 4   reviewHash      199992 non-null  object 
 5   categories      199992 non-null  object 
 6   unixReviewTime  199992 non-null  int64  
 7   itemID          199992 non-null  object 
 8   rating          199992 non-null  float64
 9   summary         199992 non-null  object 
 10  categoryID      199992 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 16.8+ MB


In [10]:
test_cat = pd.read_csv('/content/drive/My Drive/data_amazon/subdir/test_category.csv')
print('lenth of test_cat ', len(test_cat))
print('same as pairs_Category')
test_cat.head()

lenth of test_cat  13999
same as pairs_Category


,reviewTime,reviewText,helpful,reviewerID,reviewHash,unixReviewTime,rating,summary
0,2013-07-26,"I love this blouse, in fact I have it on right...","{'nHelpful': 9, 'outOf': 9}",U281659737,R934811302,1374796800,5.0,love it
1,2014-06-08,Cute product. Loved the fit. Fast shipping! I ...,"{'nHelpful': 0, 'outOf': 0}",U670561057,R657711680,1402185600,4.0,Cute but a bit uncomfortable
2,2012-03-21,I wanted a formal watch that had a big face an...,"{'nHelpful': 0, 'outOf': 0}",U433746872,R750304163,1332288000,5.0,"Bold, Large-face Watch"
3,2014-06-27,My daughter used this dress for her first comm...,"{'nHelpful': 0, 'outOf': 0}",U327816997,R865011815,1403827200,5.0,The perfect Dress
4,2014-03-21,"Nice shirt, good quality with pockets. Kind o...","{'nHelpful': 1, 'outOf': 1}",U323131234,R222729968,1395360000,5.0,Nice shirt


In [11]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
import pickle
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from wordcloud import WordCloud
import matplotlib.pyplot as plt 
from sklearn import model_selection, preprocessing

In [12]:
test_cat['reviewText'].replace(r'\b\w{1,4}\b','', regex =True, inplace = True)
train['reviewText'].replace(r'\b\w{1,4}\b','', regex =True, inplace = True)
test_cat['reviewText'].replace(np.nan,'i', regex =True, inplace = True)
train['reviewText'].replace(np.nan,'i', regex =True, inplace = True)
train.head()

,reviewTime,reviewText,helpful,reviewerID,reviewHash,categories,unixReviewTime,itemID,rating,summary,categoryID
0,2013-09-26,model picture rolled because actual...,"{'nHelpful': 0, 'outOf': 0}",U490934656,R798569390,"[['Clothing, Shoes & Jewelry', 'Women'], ['Clo...",1380153600,I402344648,4.0,High Waisted,0
1,2013-02-07,", wanted, however, about sm...","{'nHelpful': 0, 'outOf': 0}",U714157797,R436443063,"[['Clothing, Shoes & Jewelry', 'Women', 'Cloth...",1360195200,I697650540,4.0,Beautiful but size runs small,0
2,2014-03-16,comfortable wearing wedding little dia...,"{'nHelpful': 0, 'outOf': 0}",U507366950,R103439446,"[['Clothing, Shoes & Jewelry', 'Wedding Party ...",1394928000,I464613034,5.0,Great Alternative for Nurses,0
3,2014-03-10,really looks ... closer decorative ...,"{'nHelpful': 0, 'outOf': 0}",U307862152,R486351639,"[['Clothing, Shoes & Jewelry', 'Women', 'Cloth...",1394409600,I559560885,2.0,One size fits all...Questionable,0
4,2013-07-30,' quite small regular -shirt ( importan...,"{'nHelpful': 1, 'outOf': 1}",U742726598,R508664275,"[['Clothing, Shoes & Jewelry', 'Women', 'Plus-...",1375142400,I476005312,5.0,Great shirt,0


In [13]:
vectorizer = CountVectorizer()
vectorizer.fit(train['reviewText'])
vectr = vectorizer.transform(train['reviewText'])


In [14]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(vectr, train['categoryID'])

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [15]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.20, random_state=42)
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators =100, random_state=0)
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
accuracy_score(y_test, pred)

0.972120063369922

In [16]:
#vectorizer.fit(test_cat['reviewText'])
vects = vectorizer.transform(test_cat['reviewText'])

In [17]:
pred=clf.predict(vects)

In [18]:
of = test_cat[['reviewerID','reviewHash']]
of['predictions'] = pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
of.predictions.unique()

array([0, 1, 3, 4, 2])

In [20]:
of['predictions'].value_counts()

0    11596
1     2256
4       83
2       36
3       28
Name: predictions, dtype: int64

In [21]:
p = pd.DataFrame(pred)
len(p)

13999

In [22]:
clf.classes_

array([0, 1, 2, 3, 4])

In [23]:
train['categoryID'].value_counts()

0    141392
1     51414
4      2976
2      2329
3      1881
Name: categoryID, dtype: int64

In [24]:
of.to_csv("/content/drive/My Drive/data_amazon/category_predictions_updated.txt", header=None, index=None, sep=' ', mode='a')